# <span style="color:yellow"> Goldman Sachs Project</span>

## STEP 1 - SCRAPING

In [1]:
import bs4 as bs
import urllib.request as ur
import requests
import pandas as pd
import re
import codecs
import copy
import collections
import string

import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
import matplotlib

%matplotlib inline

### Creating the list of URLs

In [2]:
urls = ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32014L0065&from=EN', 
        'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32014R0596&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32017R0590&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32014R0600&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32014L0059&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32016R1712&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32012R0648&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32017R1443&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32012R0648&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32015R2365&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32019R0356&from=EN',
       'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32013R0148&from=EN']


### Getting HTML from LOOP, parsing it and saving a BS object

In [3]:
soups=[]

for i in urls: 
    #try: #tries are always halpful with url as you never know
    req = ur.Request(i)  
    html = ur.urlopen(req).read() 
    parsed = bs.BeautifulSoup(html,'lxml')
    soups.append(parsed)

### Extracting title, date and topic of the article

In [4]:
titles=[]
dates=[]
topics=[]

for i in soups: 
    #Extracting title of the article
    title = i.find('p', {"class":'doc-ti'}).text
    titles.append(title)
    
    #Extracting date when the article was published
    date = i.find('p', {"class":'hd-date'}).text[0:9]
    dates.append(date)
    
    #Extracting topic of the directive
    topic =[]
    for x in i.findAll('p',  {"class": ['doc-ti']}):
         topic.append(x.text)
    topic=topic[2]
    topics.append(topic)

This is a table that stores information on title, date and topic of each article

In [6]:
d = {'Title':titles,'Date':dates, 'Topic':topics}
df_summary = pd.DataFrame(d)
pd.set_option('display.max_colwidth', 300)
df_summary

,Title,Date,Topic
0,DIRECTIVE 2014/65/EU OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,on markets in financial instruments and amending Directive 2002/92/EC and Directive 2011/61/EU
1,REGULATION (EU) No 596/2014 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,"on market abuse (market abuse regulation) and repealing Directive 2003/6/EC of the European Parliament and of the Council and Commission Directives 2003/124/EC, 2003/125/EC and 2004/72/EC"
2,COMMISSION DELEGATED REGULATION (EU) 2017/590,31.3.2017,supplementing Regulation (EU) No 600/2014 of the European Parliament and of the Council with regard to regulatory technical standards for the reporting of transactions to competent authorities
3,REGULATION (EU) No 600/2014 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,on markets in financial instruments and amending Regulation (EU) No 648/2012
4,DIRECTIVE 2014/59/EU OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,"establishing a framework for the recovery and resolution of credit institutions and investment firms and amending Council Directive 82/891/EEC, and Directives 2001/24/EC, 2002/47/EC, 2004/25/EC, 2005/56/EC, 2007/36/EC, 2011/35/EU, 2012/30/EU and 2013/36/EU, and Regulations (EU) No 1093/2010 and ..."
5,COMMISSION DELEGATED REGULATION (EU) 2016/1712,24.9.2016,supplementing Directive 2014/59/EU of the European Parliament and of the Council establishing a framework for the recovery and resolution of credit institutions and investment firms with regard to regulatory technical standards specifying a minimum set of the information on financial contracts t...
6,REGULATION (EU) No 648/2012 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,27.7.2012,"on OTC derivatives, central counterparties and trade repositories"
7,COMMISSION IMPLEMENTING REGULATION (EU) 2017/1443,17.8.2017,amending Implementing Regulation (EU) No 680/2014 laying down implementing technical standards with regards to supervisory reporting of institutions according to Regulation (EU) No 575/2013 of the European Parliament and of the Council
8,REGULATION (EU) No 648/2012 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,27.7.2012,"on OTC derivatives, central counterparties and trade repositories"
9,REGULATION (EU) 2015/2365 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,23.12.201,on transparency of securities financing transactions and of reuse and amending Regulation (EU) No 648/2012


### Extracting all text from html

In [7]:
#Extracting texts from HTML for each link
texts=[]

for i in soups: 
    new =[]
    for x in i.findAll('p',  {"class": ['doc-sep', 'doc-end', 'hd-lg', 'separator', 'bold', 'final', 'ti-section-1', 'expanded', 'note', 'hd-date', 'signatory', 'sti-art', 'tbl-hdr', 'italic', 'normal', 'table', 'hd-oj', 'super', 'sp-normal', 'doc-ti', 'ti-section-2', 'ti-art', 'ti-tbl', 'ti-grseq-1', 'hd-ti', 'tbl-txt']}):
         new.append(x.text)
    text = "\n".join(new)
    texts.append(text)

Text of each article is save as an object of list texts. You should adress to each article by texts[i] or by using a loop.

In [10]:
# How it looks
#print(texts[2])

## STEP 2 - DATA CLEANING

### Deleting stopwords

In this part stopwords are deleted for each of the article

In [8]:
# Adding stopwords
stopwords = nltk.corpus.stopwords.words('english')
new_stopwords=['would','article','shall']
for i in new_stopwords:
    stopwords.append(i)

# Creating a function for text cleaning
def clean_text(txt):
    txt="".join([c for c in txt if c not in string.punctuation])
    tokens=re.split('\W+', txt)
    txt=[word for word in tokens if word not in stopwords]
    return txt

In [9]:
texts_no_stopwords=[]

for i in texts: 
    #Deleting stopwords
    text_no_stopwords=clean_text(i.lower())
    texts_no_stopwords.append(text_no_stopwords)

### Lemmatization: WordNet (uses a dictionary)

In this part all texts are lemmatized

In [10]:
wn=nltk.WordNetLemmatizer()

def lemmatization(token_txt):
    text=[wn.lemmatize(word) for word in token_txt]
    return text

In [11]:
texts_lemmatized=[]

for i in texts_no_stopwords: 
    #Lemmatizing texts without stopwords
    text_lemmatized=lemmatization(i)
    text_lemmatized = ' '.join([str(x) for x in text_lemmatized])
    texts_lemmatized.append(text_lemmatized)

### Making a table just for visualisation (will be deleted)

Here you may see how text changes with each step after deleting stopwords and lemmatization. Later this section will be deleted from the final code.

In [12]:
d = {'Title':titles,'Date':dates, 'Topic':topics, 'Texts':texts, 'Texts_no_stopwords':texts_no_stopwords, 'Texts_lemmatized':texts_lemmatized}
df_full = pd.DataFrame(d)
pd.set_option('display.max_colwidth', 400)
df_full.head(2)

,Title,Date,Topic,Texts,Texts_no_stopwords,Texts_lemmatized
0,DIRECTIVE 2014/65/EU OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,on markets in financial instruments and amending Directive 2002/92/EC and Directive 2011/61/EU,"12.6.2014 \nEN\nOfficial Journal of the European Union\nL 173/349\nDIRECTIVE 2014/65/EU OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 15 May 2014\non markets in financial instruments and amending Directive 2002/92/EC and Directive 2011/61/EU\n(recast)\n(Text with EEA relevance)\nTHE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION,\nHaving regard to the Treaty on the Functioning...","[1262014, en, official, journal, european, union, l, 173349, directive, 201465eu, european, parliament, council, 15, may, 2014, markets, financial, instruments, amending, directive, 200292ec, directive, 201161eu, recast, text, eea, relevance, european, parliament, council, european, union, regard, treaty, functioning, european, union, particular, 531, thereof, regard, proposal, european, commi...",1262014 en official journal european union l 173349 directive 201465eu european parliament council 15 may 2014 market financial instrument amending directive 200292ec directive 201161eu recast text eea relevance european parliament council european union regard treaty functioning european union particular 531 thereof regard proposal european commission transmission draft legislative act nation...
1,REGULATION (EU) No 596/2014 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,"on market abuse (market abuse regulation) and repealing Directive 2003/6/EC of the European Parliament and of the Council and Commission Directives 2003/124/EC, 2003/125/EC and 2004/72/EC","12.6.2014 \nEN\nOfficial Journal of the European Union\nL 173/1\nREGULATION (EU) No 596/2014 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 16 April 2014\non market abuse (market abuse regulation) and repealing Directive 2003/6/EC of the European Parliament and of the Council and Commission Directives 2003/124/EC, 2003/125/EC and 2004/72/EC\n(Text with EEA relevance)\nTHE EUROPEAN PARLIAM...","[1262014, en, official, journal, european, union, l, 1731, regulation, eu, 5962014, european, parliament, council, 16, april, 2014, market, abuse, market, abuse, regulation, repealing, directive, 20036ec, european, parliament, council, commission, directives, 2003124ec, 2003125ec, 200472ec, text, eea, relevance, european, parliament, council, european, union, regard, treaty, functioning, europ...",1262014 en official journal european union l 1731 regulation eu 5962014 european parliament council 16 april 2014 market abuse market abuse regulation repealing directive 20036ec european parliament council commission directive 2003124ec 2003125ec 200472ec text eea relevance european parliament council european union regard treaty functioning european union particular 114 thereof regard propos...


## STEP 3 - TEXT ANALYSIS

### Detailed table of most frequently used words for each article

In [18]:
# Adding word pattern
word_pattern = re.compile("^\w+$")
esw = nltk.corpus.stopwords.words('english')

# Adding function that counts length of text and tokenizes it
def get_text_counter(text):
    tokens=WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
    tokens=list(map(lambda x: x.lower(), tokens))
    tokens=[token for token in tokens if re.match(word_pattern, token) and token not in esw]
    return collections.Counter(tokens), len(tokens)

# Adding function that creates a table of most frequent words used in the text
def make_df(counter, size):
    abs_freq = np.array([el[1] for el in counter])
    rel_freq = abs_freq/size*100
    index = [el[0] for el in counter]
    df = pd.DataFrame(data=np.array([abs_freq, rel_freq]).T, index=index, columns=["Adolute Freq", "Relative Freq (%)"])
    df.index.name="Most common words"
    return df

This cell creates a table of most frequently used words in each article

In [19]:
common_word_tables=[]

for i in texts_lemmatized: 
    #Lemmatizing texts without stopwords
    t_counter, t_size=get_text_counter(i)
    common_word_table=make_df(t_counter.most_common(10), t_size)
    common_word_tables.append(common_word_table)

In [20]:
common_word_tables[5]

,Adolute Freq,Relative Freq (%)
Most common words,,
contract,46.0,3.338171
financial,40.0,2.902758
counterparty,28.0,2.031930
agreement,25.0,1.814224
authority,24.0,1.741655
margin,24.0,1.741655
resolution,22.0,1.596517
master,21.0,1.523948
regulation,20.0,1.451379


### List of most frequently used words for each article

This code extracts 10 most frequent words from the article

In [21]:
frequent_words=[]

for i in common_word_tables: 
    #Lemmatizing texts without stopwords
    frequent_word=i.index.tolist()
    frequent_words.append(frequent_word)

In [24]:
d = {'Title':titles,'Date':dates, 'Topic':topics, 'Key Words':frequent_words}
df_summary = pd.DataFrame(d)
pd.set_option('display.max_colwidth', 300)
df_summary.head()

,Title,Date,Topic,Key Words
0,DIRECTIVE 2014/65/EU OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,on markets in financial instruments and amending Directive 2002/92/EC and Directive 2011/61/EU,"[investment, member, firm, market, state, authority, client, directive, competent, service]"
1,REGULATION (EU) No 596/2014 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,"on market abuse (market abuse regulation) and repealing Directive 2003/6/EC of the European Parliament and of the Council and Commission Directives 2003/124/EC, 2003/125/EC and 2004/72/EC","[market, information, person, authority, regulation, financial, instrument, competent, trading, inside]"
2,COMMISSION DELEGATED REGULATION (EU) 2017/590,31.3.2017,supplementing Regulation (EU) No 600/2014 of the European Parliament and of the Council with regard to regulatory technical standards for the reporting of transactions to competent authorities,"[transaction, firm, investment, instrument, report, trading, code, financial, field, order]"
3,REGULATION (EU) No 600/2014 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,on markets in financial instruments and amending Regulation (EU) No 648/2012,"[trading, market, financial, venue, instrument, regulation, esma, authority, firm, competent]"
4,DIRECTIVE 2014/59/EU OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL,12.6.2014,"establishing a framework for the recovery and resolution of credit institutions and investment firms and amending Council Directive 82/891/EEC, and Directives 2001/24/EC, 2002/47/EC, 2004/25/EC, 2005/56/EC, 2007/36/EC, 2011/35/EU, 2012/30/EU and 2013/36/EU, and Regulations (EU) No 1093/2010 and ...","[resolution, authority, institution, state, member, referred, group, directive, financial, b]"


### Frequency Analysis Summary

In [32]:
tj0=df_full["Texts_lemmatized"][0]
tj1=df_full["Texts_lemmatized"][1]
tj2=df_full["Texts_lemmatized"][2]

In [35]:
tj0_counter, tj0_size=get_text_counter(tj0)
make_df(tj0_counter.most_common(20), tj0_size)
tj1_counter, tj1_size=get_text_counter(tj1)
make_df(tj1_counter.most_common(20), tj1_size)
tj2_counter, tj2_size=get_text_counter(tj2)
make_df(tj2_counter.most_common(20), tj2_size)

,Adolute Freq,Relative Freq (%)
Most common words,,
transaction,229.0,3.133552
firm,195.0,2.668309
investment,142.0,1.943076
instrument,136.0,1.860974
report,104.0,1.423098
trading,93.0,1.272578
code,88.0,1.204160
financial,85.0,1.163109
field,84.0,1.149425


In [36]:
all_counter = tj0_counter + tj2_counter + tj1_counter
all_df = make_df(tj2_counter.most_common(1000), 1)
most_common_words = all_df.index.values

In [38]:
df_data = []
for word in most_common_words:
    tj0_c = tj0_counter.get(word, 0) / tj0_size
    tj1_c = tj1_counter.get(word, 0) / tj1_size
    tj2_c = tj2_counter.get(word, 0) / tj2_size
    d = abs((tj0_c + tj1_c + tj2_c)/3)
    df_data.append([tj0_c, tj1_c, tj2_c, d])
dist_df = pd.DataFrame(data=df_data, index=most_common_words,
                       columns=["Text 0 relative frequency","Text 1 relative frequency" , "Text 2 relative frequency",
                                "Relative frequency Average"])
dist_df.index.name = "Most common words"
dist_df.sort_values("Relative frequency Average", ascending=False, inplace=True)

dist_df.head(20)



,Text 0 relative frequency,Text 1 relative frequency,Text 2 relative frequency,Relative frequency Average
Most common words,,,,
market,0.016308,0.021987,0.009442,0.015912
firm,0.016379,0.001598,0.026683,0.014887
transaction,0.003409,0.006943,0.031336,0.013896
investment,0.019503,0.002700,0.019431,0.013878
instrument,0.007296,0.011682,0.018610,0.012529
financial,0.009108,0.013446,0.011631,0.011395
authority,0.012398,0.013887,0.004789,0.010358
trading,0.008917,0.008707,0.012726,0.010116
person,0.005937,0.015209,0.009031,0.010059
